In [3]:
import numpy as np
from collections import defaultdict
import copy

# モンテカルロ法
- 前回は動的計画法を用いて状態価値関数を実装したのち,方策反復法によって最適方策を得ることができた。
    - しかし動的計画法では計算量が膨大になってしまう恐れがある、また状態遷移確率と報酬が既知である必要がある。
        - 前回実装した環境では右に行こうとすれば必ず右に行くようになっていたが、そうはならない問題も存在する。
    - そのための代替案の一つがモンテカルロ法である。


In [11]:
# 環境の実装
class GridWorld:
    def __init__(self):
        #行動は4種類(それぞれを数字に置き換える)
        self.action_list = [0,1,2,3]
        # 辞書型で行動を紐づける
        self.action_meaning = {
            0: "UP",
            1: "DOWN",
            2: "LEFT",
            3: "RIGHT"
        }
        # 以下にマップを作成する
        # それぞれの値はその場所の報酬を指す
        self.reward_map = np.array(
            [[0,0,0,1],
            [0,None,0,-1],
            [0,0,0,0]]
        )
        #以下に障害物を記載し、その座標を保持
        self.goal = (0,3)
        self.start = (2,0)
        self.wall = (1,1)
        #エージェントの場所の配列を保持(スタート値で初期化)
        self.agent_state = self.start

    # 以下にメソッドを実装
    # マップの情報を取得(主に再代入不可能のgetterの役割)
    # 高さを返却
    @property
    def height(self):
        return len(self.reward_map)
    # 横幅を返却
    @property
    def width(self):
        return len(self.reward_map[0])

    #行動の種類を返却
    @property
    def actions(self):
        return self.action_list

    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h,w)


    # 以下にエージェントを動かすための関数と報酬関数を実装する。
    def next_step(self,state,action):
        # 上下左右の行動をそれぞれ座標に足し合わせることで実装
        action_list = [(-1,0),(1,0),(0,-1),(0,1)]
        # 0~3までの数字を配列に入れることで行動となる数値を取得
        move = action_list[action]
        # エージェントを移動させる(moveを現在の状態の座標に足し合わせる)
        # next_state = (y座標,x座標)となる
        next_state = (state[0] + move[0],state[1] + move[1])
        new_y,new_x = next_state

        # 移動したのちそれがマップから外れてるかどうかの確認処理(外れていたら更新を中断)
        if new_x < 0 or new_x >= self.width or new_y < 0 or new_y >= self.height or next_state == self.wall:
            next_state = state
        return next_state #(2,3)みたいに次の座標
    def reset(self):
        self.agent_state = self.start
        return self.agent_state
    # 報酬関数
    def reward(self,state,action,next_state):
        return self.reward_map[next_state] # 1みたいに移動先での報酬を受け取る

    # 新しいコード
    # どう言った行動を起こしてどう言った報酬を受け取ったかを確認する。
    # これで行動を起こした後にどの場所に行ったかを確認できる
    def step(self, action):
        state = self.agent_state
        next_state = self.next_step(state, action)
        reward = self.reward(state, action, next_state)
        done = (next_state == self.goal)
        self.agent_state = next_state
        return next_state, reward, done


In [12]:
# 新しいモンテカルロ法用のクラスを実装
class Monte():
    def __init__(self):
        self.gamma = 0.9 # 割引率
        self.action_size = 4 # 行動の数

        random_actions = {0:0.25,1:0.25,2:0.25,3:0.25}
        self.pi = defaultdict(lambda :random_actions)
        self.V = defaultdict(lambda :0)
        self.cnts = defaultdict(lambda :0)
        self.memory = []
    def get_action(self,state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions,p=probs) # pで確率を指定できるため,ここに指定しなければ(わからない場合は)

    def add(self,state,action,reward):
        data = (state,action,reward)
        self.memory.append(data)
    def reset(self):
        self.memory.clear()
    def eval(self):
        G = 0
        for data in reversed(self.memory):
            state,action,reward = data
            G =  self.gamma * G + reward
            self.cnts[state] += 1
            self.V[state] += (G - self.V[state])/ self.cnts[state]

In [13]:
env = GridWorld()
agent = Monte()
episodes = 1000 # モンテカルロの試行回数
for episode in range(episodes):
    state = env.reset()
    agent.reset()

    while True:
        action = agent.get_action(state)
        next_state,reward,done = env.step(action)
        agent.add(state,action,reward)
        if done:
            agent.eval()
            break
        state = next_state
print(agent.V)


defaultdict(<function Monte.__init__.<locals>.<lambda> at 0x7ff240750b80>, {(1, 3): -0.37058160053012024, (1, 2): -0.5135380171882288, (0, 2): 0.18560701953442452, (0, 1): 0.08349730577296957, (0, 0): 0.015674548387005802, (1, 0): -0.02437761578173401, (2, 0): -0.10013788788646084, (2, 1): -0.21733898893290396, (2, 2): -0.4276820538360163, (2, 3): -0.7438097666642897})


In [14]:
X = np.zeros((env.width, env.height))
for i,j in agent.V.items():
    a,b = i
    X[a][b] = j
print(np.array(X))

[[ 0.01567455  0.08349731  0.18560702  0.        ]
 [-0.02437762  0.         -0.51353802 -0.3705816 ]
 [-0.10013789 -0.21733899 -0.42768205 -0.74380977]]


In [20]:
random_actions = {0:0.25,1:0.25,2:0.25,3:0.25}
pi = defaultdict(lambda :random_actions)
state = (2,3)
action_probs = pi[state]
print(action_probs)
actions = list(action_probs.keys())
actions

{0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}


[0, 1, 2, 3]